In [12]:
from selenium import webdriver
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.support.select import Select
import pandas as pd

deputados_socios_empresas = pd.read_csv("resultados/empresas_deputados.csv",sep=',',encoding = 'utf-8', converters={'cnpj': lambda x: str(x), 'cpf': lambda x: str(x), 'documento': lambda x: str(x)})

profile = webdriver.FirefoxProfile()
browser = webdriver.Firefox(profile)
browser.implicitly_wait(10)

# Site that is accessed
browser.get('https://www.fazenda.sp.gov.br/SigeoLei131/Paginas/ConsultaDespesaAno.aspx?orgao=')

# List to store the data
pagamentos = []

for num, row in deputados_socios_empresas.iterrows():
    # Variable with code to search
    empresa = (row['cnpj']).strip()

    # Search for each code in four years
    for vez in [2015, 2016, 2017, 2018]:
        ano = str(vez)

        # Select year
        Select(browser.find_element_by_name('ctl00$ContentPlaceHolder1$ddlAno')).select_by_visible_text(ano)

        # Fill in the code to search
        browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_rblDoc_0"]').click()
        browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_txtCPF"]').send_keys(empresa)
        browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_btnPesquisar"]').click()

        try:
            found = True
            alert = browser.switch_to.alert
            alert.accept()
            found = False
            # Message shows that the code was not found that year
            print("CNPJ " + empresa + " não encontrado no ano " + ano)
            browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_btnVoltar"]').click()
        except NoAlertPresentException:
            pass

        if found:
            results = browser.find_element_by_xpath("//table[@id='ctl00_ContentPlaceHolder1_gdvCredor']//tr[2]")
            cia = results.find_element_by_xpath("td[1]").text
            valor = results.find_element_by_xpath("td[2]").text

            #Message shows that the code was found that year
            print("CNPJ " + empresa + " encontrado no ano " + ano)

            # Fills dictionary with found data
            dicionario = {"cnpj": empresa,
                          "ano": ano,
                          "empresa": cia,
                          "valor": valor,
                         }
            pagamentos.append(dicionario)


            # Go back one screen to do another search
            browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_btnVoltar"]').click()

# Create the dataframe
df_pagamentos = pd.DataFrame(pagamentos)


CNPJ 08712083000153 não encontrado no ano 2015
CNPJ 08712083000153 não encontrado no ano 2016
CNPJ 08712083000153 não encontrado no ano 2017
CNPJ 08712083000153 não encontrado no ano 2018
CNPJ 10706084000137 não encontrado no ano 2015
CNPJ 10706084000137 não encontrado no ano 2016
CNPJ 10706084000137 não encontrado no ano 2017
CNPJ 10706084000137 não encontrado no ano 2018
CNPJ 05279068000175 não encontrado no ano 2015
CNPJ 05279068000175 não encontrado no ano 2016
CNPJ 05279068000175 não encontrado no ano 2017
CNPJ 05279068000175 não encontrado no ano 2018
CNPJ 96500681000126 não encontrado no ano 2015
CNPJ 96500681000126 não encontrado no ano 2016
CNPJ 96500681000126 não encontrado no ano 2017
CNPJ 96500681000126 não encontrado no ano 2018
CNPJ 07812948000190 não encontrado no ano 2015
CNPJ 07812948000190 não encontrado no ano 2016
CNPJ 07812948000190 não encontrado no ano 2017
CNPJ 07812948000190 não encontrado no ano 2018
CNPJ 03687581000170 não encontrado no ano 2015
CNPJ 03687581

CNPJ 16589618000179 não encontrado no ano 2018
CNPJ 19849453000115 não encontrado no ano 2015
CNPJ 19849453000115 não encontrado no ano 2016
CNPJ 19849453000115 não encontrado no ano 2017
CNPJ 19849453000115 não encontrado no ano 2018
CNPJ 03897704000105 não encontrado no ano 2015
CNPJ 03897704000105 não encontrado no ano 2016
CNPJ 03897704000105 não encontrado no ano 2017
CNPJ 03897704000105 não encontrado no ano 2018
CNPJ 04206374000119 não encontrado no ano 2015
CNPJ 04206374000119 não encontrado no ano 2016
CNPJ 04206374000119 não encontrado no ano 2017
CNPJ 04206374000119 não encontrado no ano 2018
CNPJ 05908763000159 não encontrado no ano 2015
CNPJ 05908763000159 não encontrado no ano 2016
CNPJ 05908763000159 não encontrado no ano 2017
CNPJ 05908763000159 não encontrado no ano 2018
CNPJ 20146015000170 não encontrado no ano 2015
CNPJ 20146015000170 não encontrado no ano 2016
CNPJ 20146015000170 não encontrado no ano 2017
CNPJ 20146015000170 não encontrado no ano 2018
CNPJ 10555395

CNPJ 05616191000134 não encontrado no ano 2017
CNPJ 05616191000134 não encontrado no ano 2018
CNPJ 21745070000149 não encontrado no ano 2015
CNPJ 21745070000149 não encontrado no ano 2016
CNPJ 21745070000149 não encontrado no ano 2017
CNPJ 21745070000149 não encontrado no ano 2018
CNPJ 21297442000111 não encontrado no ano 2015
CNPJ 21297442000111 não encontrado no ano 2016
CNPJ 21297442000111 não encontrado no ano 2017
CNPJ 21297442000111 não encontrado no ano 2018
CNPJ 44911907000188 não encontrado no ano 2015
CNPJ 44911907000188 não encontrado no ano 2016
CNPJ 44911907000188 não encontrado no ano 2017
CNPJ 44911907000188 não encontrado no ano 2018
CNPJ 49693187000145 não encontrado no ano 2015
CNPJ 49693187000145 não encontrado no ano 2016
CNPJ 49693187000145 não encontrado no ano 2017
CNPJ 49693187000145 não encontrado no ano 2018
CNPJ 21888741000120 não encontrado no ano 2015
CNPJ 21888741000120 não encontrado no ano 2016
CNPJ 21888741000120 não encontrado no ano 2017
CNPJ 21888741

CNPJ 78306214000188 não encontrado no ano 2016
CNPJ 78306214000188 não encontrado no ano 2017
CNPJ 78306214000188 não encontrado no ano 2018
CNPJ 05849495000141 não encontrado no ano 2015
CNPJ 05849495000141 não encontrado no ano 2016
CNPJ 05849495000141 não encontrado no ano 2017
CNPJ 05849495000141 não encontrado no ano 2018
CNPJ 07022473000139 encontrado no ano 2015
CNPJ 07022473000139 encontrado no ano 2016
CNPJ 07022473000139 encontrado no ano 2017
CNPJ 07022473000139 encontrado no ano 2018
CNPJ 52717766000195 não encontrado no ano 2015
CNPJ 52717766000195 não encontrado no ano 2016
CNPJ 52717766000195 não encontrado no ano 2017
CNPJ 52717766000195 não encontrado no ano 2018
CNPJ 54143755000174 não encontrado no ano 2015
CNPJ 54143755000174 não encontrado no ano 2016
CNPJ 54143755000174 não encontrado no ano 2017
CNPJ 54143755000174 não encontrado no ano 2018
CNPJ 16384854000159 não encontrado no ano 2015
CNPJ 16384854000159 não encontrado no ano 2016
CNPJ 16384854000159 não encon

CNPJ 00177136000171 não encontrado no ano 2015
CNPJ 00177136000171 não encontrado no ano 2016
CNPJ 00177136000171 não encontrado no ano 2017
CNPJ 00177136000171 não encontrado no ano 2018
CNPJ 57353583000151 não encontrado no ano 2015
CNPJ 57353583000151 não encontrado no ano 2016
CNPJ 57353583000151 não encontrado no ano 2017
CNPJ 57353583000151 não encontrado no ano 2018
CNPJ 08245853000103 não encontrado no ano 2015
CNPJ 08245853000103 não encontrado no ano 2016
CNPJ 08245853000103 não encontrado no ano 2017
CNPJ 08245853000103 não encontrado no ano 2018
CNPJ 03442645000172 não encontrado no ano 2015
CNPJ 03442645000172 não encontrado no ano 2016
CNPJ 03442645000172 não encontrado no ano 2017
CNPJ 03442645000172 não encontrado no ano 2018
CNPJ 17337055000194 não encontrado no ano 2015
CNPJ 17337055000194 não encontrado no ano 2016
CNPJ 17337055000194 não encontrado no ano 2017
CNPJ 17337055000194 não encontrado no ano 2018
CNPJ 17754517000179 não encontrado no ano 2015
CNPJ 17754517

CNPJ 49978158000120 não encontrado no ano 2018
CNPJ 07592608000100 não encontrado no ano 2015
CNPJ 07592608000100 não encontrado no ano 2016
CNPJ 07592608000100 não encontrado no ano 2017
CNPJ 07592608000100 não encontrado no ano 2018
CNPJ 08095624000141 não encontrado no ano 2015
CNPJ 08095624000141 encontrado no ano 2016
CNPJ 08095624000141 não encontrado no ano 2017
CNPJ 08095624000141 não encontrado no ano 2018
CNPJ 09270255000149 não encontrado no ano 2015
CNPJ 09270255000149 não encontrado no ano 2016
CNPJ 09270255000149 não encontrado no ano 2017
CNPJ 09270255000149 não encontrado no ano 2018
CNPJ 19487984000105 não encontrado no ano 2015
CNPJ 19487984000105 não encontrado no ano 2016
CNPJ 19487984000105 não encontrado no ano 2017
CNPJ 19487984000105 não encontrado no ano 2018
CNPJ 19488002000108 não encontrado no ano 2015
CNPJ 19488002000108 não encontrado no ano 2016
CNPJ 19488002000108 não encontrado no ano 2017
CNPJ 19488002000108 não encontrado no ano 2018
CNPJ 112016140001

In [13]:
df_pagamentos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
ano        5 non-null object
cnpj       5 non-null object
empresa    5 non-null object
valor      5 non-null object
dtypes: object(4)
memory usage: 240.0+ bytes


In [14]:
df_pagamentos.reset_index()

,index,ano,cnpj,empresa,valor
0,0,2015,07022473000139,07022473000139 - RAPIDO FENIX VIACAO LTDA,"1.275,90"
1,1,2016,07022473000139,07022473000139 - RAPIDO FENIX VIACAO LTDA,"1.027,50"
2,2,2017,07022473000139,07022473000139 - RAPIDO FENIX VIACAO LTDA,"1.125,36"
3,3,2018,07022473000139,07022473000139 - RAPIDO FENIX VIACAO LTDA,"3.668,20"
4,4,2016,08095624000141,08095624000141 - BONANZANET INFORM AVAL BENS E...,"1.390,00"


In [15]:
df_pagamentos.to_csv('resultados/pagtos_estaduais_empresas_deputados.csv', index=False, decimal=',')